In [1]:
import pandas as pd
import numpy as np
from config.config import OUTPUT_CSV_FILE, CLEANED_CSV_FILE

df = pd.read_csv(CLEANED_CSV_FILE)
df

,Unnamed: 0,amount,city,marka,model,year,type,color,distance,motor_volume,horse_power,motor_type,is_outlier
0,0,17500.0,Bakı,Hyundai,Sonata,2010,Sedan,Gümüşü,222000,2.4,178,Benzin,False
1,1,27300.0,Bakı,Toyota,Corolla,2021,Sedan,Qara,73000,1.6,122,Benzin,False
2,2,32123.0,Bakı,Toyota,Corolla,2024,Sedan,Ağ,0,1.8,98,Hibrid,False
3,3,17500.0,Qazax,Chevrolet,Trax,2018,"Offroader / SUV, 5 qapı",Qara,205000,1.4,138,Benzin,False
4,4,24900.0,Bakı,Ford,Fusion (North America),2016,Sedan,Ağ,169000,2.0,240,Plug-in Hibrid,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3060,3060,18800.0,Bakı,Ford,Transit,2007,Furqon,Ağ,195000,2.2,125,Dizel,False
3061,3061,36000.0,Bakı,Volkswagen,Touareg,2015,"Offroader / SUV, 5 qapı",Ağ,129000,3.6,249,Benzin,False
3062,3062,14800.0,Bakı,Nissan,March,2016,"Hetçbek, 5 qapı",Tünd qırmızı,54000,1.2,79,Benzin,False
3063,3063,12500.0,Bakı,Hyundai,i30,2010,"Hetçbek, 5 qapı",Qara,260000,1.4,109,Benzin,False
